In [ ]:
#r "./../../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,Password,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,PrivateKeyFilePath,AdditionalBatchCommands,DeploymentBaseDirectoryAtRemote,SlurmAccount,Email,MonoDebug
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,C:\Users\weber\AppData\Local\BoSSS-LocalJobs,False,LocalPC,dotnet,<null>,,,,,,,,,,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\weber\dply,True,FDY-cluster,dotnet,,"[ { \\hpccluster\hpccluster-scratch\weber == : LocalMountPath: \\hpccluster\hpccluster-scratch\weber, PathAtRemote: } ]",FDY\weber,<null>,DC2,[ HPCCLUSTER2 ],Normal,True,,,,,,
2,BoSSS.Application.BoSSSpad.SlurmClient,X:\Deployerie,True,<null>,dotnet,,,jw52xeqa,<null>,lcluster5.hrz.tu-darmstadt.de,,,,C:\Users\weber\.ssh\id_rsa,<null>,/home/jw52xeqa/Deployerie,project01217,<null>,False
3,BoSSS.Application.BoSSSpad.SlurmClient,X:\Deployerie,True,HHLR,dotnet,,"[ { W:\work\scratch\jw52xeqa == /work/scratch/jw52xeqa: LocalMountPath: W:\work\scratch\jw52xeqa, PathAtRemote: /work/scratch/jw52xeqa } ]",jw52xeqa,<null>,lcluster14.hrz.tu-darmstadt.de,,,,C:\Users\weber\.ssh\id_rsa,[ #SBATCH -p test24 ],/home/jw52xeqa/Deployerie,special00006,<null>,False


In [ ]:
using BoSSS.Application.BoSSSpad;

In [ ]:
// ==================================
// setup Client & Workflow & Database
// ==================================
var myBatch = (SlurmClient)ExecutionQueues[3];
//myBatch.AdditionalBatchCommands = new string[]{"#SBATCH -p test24","#SBATCH -C avx512"}; //new string[]{"#SLURM -p test24", "#SLURM -C avx512","#SLURM -N 3"}; 

In [ ]:

string WFlowName = "IBM_testKarmanVortexStreet";
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

In [ ]:
string dirname = "DB_IBM_test";
string winpath = @"H:\weber\"+dirname;
var tempDB = OpenOrCreateDatabase(winpath);

In [ ]:
tempDB.Sessions

#0: IBM_RotSphere	XNSE_rotsphere*	08/26/2021 21:05:49	b0eeec16...
#1: IBM_RotSphere	XNSE_rotsphere*	08/26/2021 20:57:46	b33248de...
#2: IBM_RotSphere	XNSE_rotsphere*	08/26/2021 20:21:33	9b723578...
#3: IBM_RotSphere	XNSE_rotsphere*	08/26/2021 19:59:53	be8ce2a3...
#4: IBM_RotSphere	XNSE_rotsphere*	08/26/2021 14:53:48	22034391...
#5: IBM_RotSphere	XNSE_rotsphere	08/26/2021 13:29:09	217c6547...
#6: IBM_RotSphere	XNSE_rotsphere*	08/26/2021 13:21:35	da4fcea9...
#7: IBM_RotSphere	XNSE_rotsphere*	08/26/2021 13:21:09	637db658...
#8: XNSE/IBM_benchmark	XNSE_rotsphere*	08/25/2021 09:45:48	827d43c8...
#9: XNSE/IBM_benchmark	XNSE_rotsphere*	08/25/2021 07:18:42	1b8831b5...
#10: XNSE/IBM_benchmark	XNSE_rotsphere	08/20/2021 16:23:04	c356cdc0...
#11: XNSE/IBM_benchmark	XNSE_rotsphere	08/20/2021 13:06:13	a9be193a...
#12: XNSE/IBM_benchmark	XNSE_rotsphere*	08/20/2021 12:27:28	a0923f22...
#13: XNSE/IBM_benchmark	XNSE_rotsphere*	08/20/2021 11:56:29	dc7e34fc...
#14: XNSE/IBM_benchmark	XNSE_rotsphere*	08/20

In [ ]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
}

In [ ]:
// set parameterz
int[] core_sweep = {16};
int p             = 2;
int Problemsize   = 250000;
int MemoryPerCore = 2000;

In [ ]:
// calculate grid size
double Res    = Math.Pow(Problemsize/(Utils.Np(p)*3+Utils.Np(p-1)),0.333333333);
int ResRndup  = (int)Math.Ceiling(Res);
int NoOfCells = ResRndup*ResRndup*ResRndup;
double test   = NoOfCells*(Utils.Np(p)*3+Utils.Np(p-1));

In [ ]:
ResRndup

20

In [ ]:
List<string> controls = new List<string>();

In [ ]:
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;

In [ ]:
int[] PolyDegS = new int[] {p};

In [ ]:
// --------------------
// setup control object
// --------------------

//foreach(LinearSolverCode solver_name in solver_nameS) {
    foreach(int k in PolyDegS) { 
        //Rotating_Cube(int k = 3, int Res = 20, int SpaceDim = 2, bool useAMR = false, int NoOfTimesteps = 1,bool writeToDB = false, bool tracing = false, bool loadbalancing = false)
        //string jobstr = String.Format(@"cs:BoSSS.Application.XNSE_Solver.HardcodedControl.Rotating_Cube({0}, {1}, {2},true,{3},true,true,false)",2,ResRndup,3,10);
        //string jobstr = String.Format(@"cs:BoSSS.Application.XNSE_Solver.HardcodedControl.Rotating_Sphere({0}, {1}, {2},true,{3},true,true,true)",2,ResRndup,3,10);
        //KarmanVortexStreet(int k=1, int Res=20, int SpaceDim=2, int NoOfTimeSteps=100 ,bool UseAMR=true, bool writeToDB=true, bool loadbalancing=true)
        string jobstr = String.Format(@"cs:BoSSS.Application.XNSE_Solver.HardcodedControl.KarmanVortexStreet({0}, {1}, {2},{3},true,true,true)",3,ResRndup,2,100);
        controls.Add(jobstr);
        Console.WriteLine(jobstr);
    }

In [ ]:
controls

index,value
0,"cs:BoSSS.Application.XNSE_Solver.HardcodedControl.KarmanVortexStreet(3, 20, 2,100,true,true,true)"


In [ ]:
// ==========
// Launch Job
// ==========
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;

In [ ]:
foreach(var cores in core_sweep){
    foreach(var ctrlstr in controls){
        int rndnum = new Random().Next();
        var aJob   = new Job("rotting_sphere_"+cores+rndnum,typeof(XNSE));
        aJob.SetControlStatement(ctrlstr);
        aJob.NumberOfMPIProcs         = cores;
        aJob.ExecutionTime            = "4:00:00";
        aJob.MemPerCPU                = "4000";
        aJob.UseComputeNodesExclusive = true;
        aJob.Activate(myBatch);
    }
}